In [35]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
tokenizer = T5Tokenizer.from_pretrained('t5-small')
# input = "predict tail: barack obama | position_held |"
# input = "translate English to German: How are you doing?"
input = "predict tail: malawi | member of |"
model = T5ForConditionalGeneration.from_pretrained('models/checkpoint_codex_base_15000.pt')
input_ids = tokenizer(input, return_tensors="pt").input_ids  # Batch size 1
outputs = model.generate(input_ids)

In [36]:
print(input)
print(''.join(tokenizer.convert_ids_to_tokens(outputs[0])))

predict tail: malawi | member of |
<pad>▁international▁development▁association</s>


In [34]:
input_ids

tensor([[9689, 9891,   10,  954, 4207,   23, 1820, 1144,   13, 1820,    1]])

In [46]:
from transformers import (
    HfArgumentParser,
    DataCollator,
    Trainer,
    TrainingArguments,
    set_seed,
)


In [3]:
outputs[0]

tensor([   0, 1038,  606, 6028,    1])

In [26]:
actual = tokenizer("international development association", return_tensors="pt").input_ids[0].numpy()

In [27]:
actual

array([1038,  606, 6028,    1])

In [28]:
predicted = outputs[0][1:].numpy()

In [29]:
predicted

array([1038,  606, 6028,    1])

In [30]:
actual == predicted

array([ True,  True,  True,  True])

In [25]:
actual.numpy()

array([1038,  606, 6028,    1])